In [5]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = [0]*300
    yc = [0]*300
    maxrows = 300
    maxcols = 300
    Cost = [([0] * maxcols) for row in range(maxrows) ]

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc[it-1] = float(tokens[0])
        yc[it-1] = float(tokens[1])
    
    for i in range(0, dim):
        for j in range(0, dim):
            Cost[i][j] = pow((pow(xc[i]-xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
        
    for i in range(0, dim):
        for j in range(0, dim):
            if(i==j):
                Cost[i][j] = 99999
    return Cost, xc, yc, dim

In [7]:
from gurobipy import *
import string
import time

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_5_s_1.dat")
Cost, xc, yc, dim = Preprocess(f)

startTime = time.time()
TSP_MTZ = Model('TSP_MTZ')

x = [[0 for i in range(dim)] for j in range(dim)] 
for i in range(0, dim):
    for j in range(0, dim):
        x[i][j] = TSP_MTZ.addVar(vtype=GRB.BINARY, name = "x[][]".format(i,j))

u = [i for i in range(0, dim)]
for i in range(0, dim):
    u[i] = TSP_MTZ.addVar(vtype=GRB.INTEGER, name = 'u[]'.format(i))
    
TSP_MTZ.update()

TSP_MTZ.setObjective(quicksum(quicksum(Cost[i][j]*x[i][j] for i in range(dim)) for j in range(dim)), GRB.MINIMIZE)



for i in range(0, dim):
    TSP_MTZ.addConstr(quicksum(x[i][j] for j in range(0, dim) ) == 1)
    TSP_MTZ.addConstr(quicksum(x[j][i] for j in range(0, dim) ) == 1)
    
for i in range(0, dim):
    for j in range(0, dim):
        if(i!=0 and j!=0):
            TSP_MTZ.addConstr(u[i] - u[j] + dim*x[i][j] <= dim-1)


TSP_MTZ.optimize()


executionTime = (time.time() - startTime)
print("The execution time is")
print(executionTime)

############################          Contstruct MTZ Tour       ######################################################
print("The Tour is:   ", end= " ")
i = 0

print("1", end= " ")
for iter1 in range(0, dim):
    for j in range(0, dim):
        if(x[i][j].x == 1.0):
            print("->", end = " ")
            print(j+1, end=" ")
            break
    i = j



Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 26 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x766bbac8
Variable types: 0 continuous, 30 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [3e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 46.8862227
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolved: 22 rows, 24 columns, 76 nonzeros
Variable types: 0 continuous, 24 integer (20 binary)

Root relaxation: objective 4.492813e+01, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   46.02976    0   10   46.88622   46.02976  1.83%     -    0s
H    0     0                      46.0297578   46.02976  0.00%     -    0s
     0     0   46.02976    0   10   46.02976   46.02976  0.00%     - 